In [44]:
#read in json file into pandas
import pandas as pd
df = pd.read_json('demo_data.json')

In [47]:
import datetime

df['time'] = df['time'].apply(lambda x: datetime.datetime.fromtimestamp(x / 1000.0))

In [38]:
import pickle
#load model
model = pickle.load(open('xgb_model.pkl', 'rb'))

In [48]:
df.head()

,location_id,time,temperature_2m,relative_humidity_2m,rain,pressure_msl,surface_pressure,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,soil_temperature_0_to_7cm,county_name,wind_shear
0,0,2024-05-20 19:00:00,61.7,88,0.272,1007.7,958.3,15.6,25.5,35,35,64.4,Lyon,9.900000
1,0,2024-05-20 20:00:00,61.2,89,0.008,1006.6,957.2,13.1,22.1,37,39,63.1,Lyon,9.019574
2,0,2024-05-20 21:00:00,60.4,91,0.000,1005.6,956.2,13.1,22.8,57,60,62.2,Lyon,9.742108
3,0,2024-05-20 22:00:00,60.1,93,0.000,1005.0,955.6,11.1,19.6,54,59,61.7,Lyon,8.596846
4,0,2024-05-20 23:00:00,59.8,95,0.000,1004.0,954.6,10.8,19.1,68,72,61.4,Lyon,8.360321


In [49]:
#make a df for each unique time stamp
df=df[df['time']=='2024-05-20 22:00:00']

In [22]:
df.columns

Index(['ï»¿location_id', 'time', 'temperature_2m', 'relative_humidity_2m',
       'rain', 'pressure_msl', 'surface_pressure', 'wind_speed_10m',
       'wind_speed_100m', 'wind_direction_10m', 'wind_direction_100m',
       'soil_temperature_0_to_7cm', 'wind_shear'],
      dtype='object')

In [50]:
features=['temperature_2m', 'relative_humidity_2m',
       'rain', 'pressure_msl', 'surface_pressure', 'wind_speed_10m',
       'wind_speed_100m', 'wind_direction_10m', 'wind_direction_100m',
       'soil_temperature_0_to_7cm', 'wind_shear']

In [51]:
X=df[features]

In [53]:
#Make a dataframe that has Time | County | Prediction that is empty
df_st= pd.DataFrame(columns=['Time', 'County', 'Prediction'])

In [54]:
df.head()

,location_id,time,temperature_2m,relative_humidity_2m,rain,pressure_msl,surface_pressure,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,soil_temperature_0_to_7cm,county_name,wind_shear
3,0,2024-05-20 22:00:00,60.1,93,0.0,1005.0,955.6,11.1,19.6,54,59,61.7,Lyon,8.596846
27,1,2024-05-20 22:00:00,59.7,94,0.0,1005.1,950.1,13.2,22.3,58,61,62.4,Osceola,9.144223
51,2,2024-05-20 22:00:00,61.0,98,0.0,1005.5,956.8,11.6,19.2,56,58,64.3,Dickinson,7.617831
75,3,2024-05-20 22:00:00,61.2,97,0.0,1005.6,960.0,10.0,16.6,40,48,64.3,Emmet,6.840395
99,4,2024-05-20 22:00:00,62.7,85,0.0,1008.6,973.0,8.0,16.3,142,148,63.6,Allamakee,8.385624


In [58]:
#for each record in the df, make a prediction and append to df_st. proba off features X, df[Prediction] off model
max_time= df['time'].max()
df_st= pd.DataFrame(columns=['Time', 'County', 'Prediction'])

#for each unique county, get the latest information from df, store features in X, make a prediction, append to df_st
for county in df['county_name'].unique():
    X=df[df['county_name']==county][features]
    pred=model.predict_proba(X)
    df_st=df_st.append({'Time':max_time, 'County':county, 'Prediction':pred[0][1]}, ignore_index=True)

AttributeError: 'DataFrame' object has no attribute 'append'